<a href="https://colab.research.google.com/github/jmestanza/time_stretching/blob/master/speed_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%cd /content
from IPython.display import clear_output
#!git clone https://github.com/jmestanza/time_stretching.git

#imports
!pip install webrtcvad
import webrtcvad
#!pip install librosa
!pip install --upgrade librosa
import librosa 
import scipy
from IPython.display import Audio

In [0]:
#local modules
!wget https://raw.githubusercontent.com/jmestanza/time_stretching/master/LPC%2C%20PSOLA%20and%20Pitch%20detection%20speed%20up/detect_pitch.py
!wget https://raw.githubusercontent.com/jmestanza/time_stretching/master/LPC%2C%20PSOLA%20and%20Pitch%20detection%20speed%20up/modified_psola.py

from modified_psola import modified_psola
from detect_pitch import get_fundamental_frequency

#audios
!wget "https://raw.githubusercontent.com/jmestanza/time_stretching/master/LPC%2C%20PSOLA%20and%20Pitch%20detection%20speed%20up/input/ahh_without_silences_8k.wav"
!wget "https://raw.githubusercontent.com/jmestanza/time_stretching/master/LPC%2C%20PSOLA%20and%20Pitch%20detection%20speed%20up/input/ahh_without_silences_8k_norm.wav"

clear_output()

In [15]:
fs = 8000

#os.chdir("/content/time_stretching/code/input/")

audio_file = "/content/ahh_without_silences_8k_norm.wav"
ahh, _ = librosa.load(audio_file, sr=fs)


# necesito al menos dos periodos de la señal
#200 = (50+25)+50+(50+25)
#L = 50 # muestras de la ventana
#K = L//2
# tamaño total por observacion
# total = 4*L

#quiero tener 20 milisegundos de señal
T = 20*1e-3
samples = int(T * fs) 
L = samples//4
K = L

a = librosa.lpc(ahh, 12)
e_ = scipy.signal.lfilter(a,[1], ahh) # obtenemos la señal error a traves del filtrado

f0,sample_f0 = get_fundamental_frequency(e_,K,L,fs)

print("Frecuencia fundamental : ",f0," ", sample_f0)
speed = 1.0
error_audio = modified_psola(e_,sample_f0, speed)

new_audio = scipy.signal.lfilter([1],a, error_audio)

#os.chdir("../output")
#librosa.output.write_wav("e_"+audio_file+f"_speed_x{speed}.wav", new_audio, fs)

Audio(data=new_audio,rate=fs)



Frecuencia fundamental :  136.7477474849691   59
